In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os, requests
from matplotlib import colors
from matplotlib import rcParams
from matplotlib.colors import ListedColormap
from numpy import pi
from copy import copy

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score

from scipy.stats import poisson
from sklearn.metrics import mean_squared_error
import statsmodels.api as sm

In [2]:
from scipy.stats import vonmises
from scipy.optimize import minimize
from scipy.special import i0
from scipy.integrate import quad
from scipy.signal import fftconvolve

In [3]:
# @title Data retrieval

url = "https://github.com/steevelaquitaine/projInference/raw/gh-pages/data/csv/data01_direction4priors.csv"
try:
    RequestAPI = requests.get(url)
except requests.ConnectionError:
    print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
else:
    if RequestAPI.status_code != requests.codes.ok:
        print("Failed to download data. Please contact steeve.laquitaine@epfl.ch")
    else:
        with open("data01_direction4priors.csv", "wb") as fid:
            fid.write(RequestAPI.content)

In [4]:
# @title Data loading
data = pd.read_csv("data01_direction4priors.csv")
data = pd.DataFrame(data)

In [41]:
class BayesianObserverModel:
    def __init__(
        self,
        kappa_e_low,
        kappa_e_med,
        kappa_e_high,
        kappa_prior_low,
        kappa_prior_med,
        kappa_prior_high,
        kappa_prior_high_coh,
        kappa_m,
        p_r,
    ):
        # Three likelihood strengths that vary with motion coherence
        self.kappa_e_low = kappa_e_low
        self.kappa_e_med = kappa_e_med
        self.kappa_e_high = kappa_e_high

        # Four prior strengths that vary with experimental prior strength
        self.kappa_prior_low = kappa_prior_low
        self.kappa_prior_med = kappa_prior_med
        self.kappa_prior_high = kappa_prior_high
        self.kappa_prior_high_coh = kappa_prior_high_coh

        # Motor noise strength
        self.kappa_m = kappa_m

        # Probability of random estimation (lapse rate)
        self.p_r = p_r

        # Define the resolution for circular calculations
        self.n_bins = 360
        self.theta_range = np.linspace(0, 2 * np.pi, self.n_bins, endpoint=False)

    def get_kappa_e(self, coherence):
        if coherence == 0.06:
            return self.kappa_e_low
        elif coherence == 0.12:
            return self.kappa_e_med
        else:
            return self.kappa_e_high

    def get_kappa_prior(self, prior_std, coherence):
        if prior_std == 10:
            return self.kappa_prior_low
        elif prior_std == 20:
            return self.kappa_prior_med
        elif prior_std == 40:
            return self.kappa_prior_high
        else:
            return self.kappa_prior_high_coh

    def compute_unnormalized_posterior(
        self, theta_true, theta_est, coherence, prior_std, prior_mean
    ):
        kappa_e = self.get_kappa_e(coherence)
        kappa_prior = self.get_kappa_prior(prior_std, coherence)

        # Compute likelihood
        likelihood = vonmises.pdf(theta_est, loc=theta_true, kappa=kappa_e)

        # Compute prior
        prior = vonmises.pdf(theta_true, loc=prior_mean, kappa=kappa_prior)

        # Compute posterior
        unnormalized_posterior = likelihood * prior

        return unnormalized_posterior

    def compute_percept(self, theta_true, theta_est, coherence, prior_std, prior_mean):
        # Not sure if we should sum over theta_est or theta_true
        marginal_likelihood = np.sum(
            [
                self.compute_unnormalized_posterior(
                    theta_true, theta_est, coherence, prior_std, prior_mean
                )
                for theta_est in self.theta_range
            ]
        )
        posteriors = [
            (
                self.compute_unnormalized_posterior(
                    theta_true, theta_est, coherence, prior_std, prior_mean
                )
                / marginal_likelihood
            )
            for theta_true in self.theta_range
        ]
        return self.theta_range[np.argmax(posteriors)]

    def compute_percept_distribution(
        self, theta_true, coherence, prior_std, prior_mean
    ):
        percept_dist = np.zeros(self.n_bins)

        for theta_est in self.theta_range:
            theta_p = self.compute_percept(
                theta_true, theta_est, coherence, prior_std, prior_mean
            )
            percept_bin = (
                int(np.round(theta_p / (2 * np.pi) * self.n_bins)) % self.n_bins
            )
            percept_dist[percept_bin] += vonmises.pdf(
                theta_est, loc=theta_true, kappa=self.get_kappa_e(coherence)
            )

        # Normalize the percept distribution
        percept_dist /= np.sum(percept_dist)

        return percept_dist

    def circular_convolve(self, x, y):
        return fftconvolve(x, y, mode="same") / sum(y)

    def generate_estimate_distribution(
        self, theta_true, theta_est, coherence, prior_std, prior_mean
    ):
        percept_dist = self.compute_percept_distribution(
            theta_true, coherence, prior_std, prior_mean
        )

        # Motor noise distribution
        motor_noise = vonmises.pdf(self.theta_range, loc=0, kappa=self.kappa_m)

        # Convolve posterior with motor noise
        estimate_dist = self.circular_convolve(percept_dist, motor_noise)

        # Incorporate lapse rate
        uniform_dist = np.ones(self.n_bins) / self.n_bins
        estimate_dist = (1 - self.p_r) * estimate_dist + self.p_r * uniform_dist

        return estimate_dist

    def log_likelihood(self, params, data):
        (
            self.kappa_e_low,
            self.kappa_e_med,
            self.kappa_e_high,
            self.kappa_prior_low,
            self.kappa_prior_med,
            self.kappa_prior_high,
            self.kappa_prior_high_coh,
            self.kappa_m,
            self.p_r,
        ) = params
        log_prob = 0
        for theta_true, theta_est, coherence, prior_std, prior_mean in data:
            # We fit to subject estimates
            estimate_dist = self.generate_estimate_distribution(
                theta_true, theta_est, coherence, prior_std, prior_mean
            )
            plt.plot(estimate_dist)
            # Find the bin corresponding to theta_est
            est_bin = int(np.round(theta_est / (2 * np.pi) * self.n_bins)) % self.n_bins
            log_prob += np.log(
                estimate_dist[est_bin] + 1e-10
            )  # Add small constant to avoid log(0)
        return -log_prob  # Return negative log-likelihood for minimization

    def fit(self, data):
        # Initial parameter guesses
        initial_params = [1.0, 2.0, 3.0, 0.5, 1.0, 1.5, 2.0, 1.0, 0.1]

        # Fit the model using maximum likelihood estimation
        result = minimize(
            self.log_likelihood, initial_params, args=(data,), method="Nelder-Mead"
        )

        (
            self.kappa_e_low,
            self.kappa_e_med,
            self.kappa_e_high,
            self.kappa_prior_low,
            self.kappa_prior_med,
            self.kappa_prior_high,
            self.kappa_prior_high_coh,
            self.kappa_m,
            self.p_r,
        ) = result.x
        return result.x


# Usage example:
# data = [(true_direction, estimated_direction, coherence, prior_std, prior_mean), ...]
# model = BayesianObserverModel(kappa_e_low=1, kappa_e_med=2, kappa_e_high=3,
#                               kappa_prior_low=0.5, kappa_prior_med=1, kappa_prior_high=1.5, kappa_prior_high_coh=2,
#                               kappa_m=1, p_r=0.1)
# fitted_params = model.fit(data)
# print("Fitted parameters:", fitted_params)

In [42]:
test_model = BayesianObserverModel(
    kappa_e_low=1,
    kappa_e_med=2,
    kappa_e_high=3,
    kappa_prior_low=0.5,
    kappa_prior_med=1,
    kappa_prior_high=1.5,
    kappa_prior_high_coh=2,
    kappa_m=1,
    p_r=0.1,
)

test_model.log_likelihood((1, 2, 3, 0.5, 1, 1.5, 2, 1, 0.1), model_data)

KeyboardInterrupt: 

In [ ]:
# Displayed direction is theta true
# Estimated direction is theta est

In [14]:
data.head()

,trial_index,trial_time,response_arrow_start_angle,motion_direction,motion_coherence,estimate_x,estimate_y,reaction_time,raw_response_time,prior_std,prior_mean,subject_id,experiment_name,experiment_id,session_id,run_id
0,1,0.000000,NaN,225,0.12,-1.749685,-1.785666,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
1,2,2.730730,NaN,225,0.12,-1.819693,-1.714269,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
2,3,4.913950,NaN,235,0.06,-1.562674,-1.951422,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
3,4,6.997296,NaN,225,0.06,-1.601388,-1.919781,NaN,NaN,10,225,1,data01_direction4priors,11,1,1
4,5,9.097130,NaN,215,0.24,-1.639461,-1.887371,NaN,NaN,10,225,1,data01_direction4priors,11,1,1


In [9]:
# @title Basic plots of subject estimate average by task conditions

# circular statistics utils
# -------------------
def get_cartesian_to_deg(x: np.ndarray, y: np.ndarray, signed: bool) -> np.ndarray:
    """convert cartesian coordinates to
    angles in degree
    Args:
        x (np.ndarray): x coordinate
        y (np.ndarray): y coordinate
        signed (boolean): True (signed) or False (unsigned)
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_cartesian_to_deg
            x = np.array([1, 0, -1, 0])
            y = np.array([0, 1, 0, -1])
            degree = get_cartesian_to_deg(x,y,False)
            # Out: array([  0.,  90., 180., 270.])
    Returns:
        np.ndarray: angles in degree
    """
    # convert to radian (ignoring divide by 0 warning)
    with np.errstate(divide="ignore"):
        degree = np.arctan(y / x)

    # convert to degree and adjust based
    # on quadrant
    for ix in range(len(x)):
        if (x[ix] >= 0) and (y[ix] >= 0):
            degree[ix] = degree[ix] * 180 / np.pi
        elif (x[ix] == 0) and (y[ix] == 0):
            degree[ix] = 0
        elif x[ix] < 0:
            degree[ix] = degree[ix] * 180 / np.pi + 180
        elif (x[ix] >= 0) and (y[ix] < 0):
            degree[ix] = degree[ix] * 180 / np.pi + 360

    # if needed, convert signed to unsigned
    if not signed:
        degree[degree < 0] = degree[degree < 0] + 360
    return degree


def get_deg_to_rad(deg: np.array, signed: bool):
    """convert angles in degree to radian
    Args:
        deg (np.array): angles in degree
        signed (bool): True (signed) or False (unsigned)
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_deg_to_rad
            radians = get_deg_to_rad(np.array([0, 90, 180, 270], True)
            Out: array([ 0., 1.57079633, 3.14159265, -1.57079633])
    Returns:
        np.ndarray: angles in radian
    """
    # get unsigned radians (1:2*pi)
    rad = (deg / 360) * 2 * pi

    # get signed radians(-pi:pi)
    if signed:
        rad[deg > 180] = (deg[deg > 180] - 360) * (2 * pi / 360)
    return rad


def get_polar_to_cartesian(angle: np.ndarray, radius: float, type: str) -> dict:
    """convert angle in degree or radian to cartesian coordinates
    Args:
        angle (np.ndarray): angles in degree or radian
        radius (float): radius
        type (str): "polar" or "radian"
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_polar_to_cartesian
            degree = np.array([0, 90, 180, 270])
            cartesian = get_polar_to_cartesian(degree, 1, "polar")
            cartesian.keys()

            # Out: dict_keys(['deg', 'rad', 'cart'])

            cartesian["cart"]

            # Out: array([[ 1.,  0.],
            #            [ 0.,  1.],
            #            [-1.,  0.],
            #            [-0., -1.]])
    Returns:
        dict: _description_
    """
    # convert to radian if needed
    theta = dict()
    if type == "polar":
        theta["deg"] = angle
        theta["rad"] = angle * np.pi / 180
    elif type == "radian":
        theta["deg"] = get_deg_to_rad(angle, False)
        theta["rad"] = angle

    # convert to cartesian coordinates
    x = radius * np.cos(theta["rad"])
    y = radius * np.sin(theta["rad"])

    # round to 10e-4
    x = np.round(x, 4)
    y = np.round(y, 4)

    # reshape as (N angles x 2 coord)
    theta["cart"] = np.vstack([x, y]).T
    return theta


def get_circ_weighted_mean_std(angle: np.ndarray, proba: np.ndarray, type: str) -> dict:
    """calculate circular data statistics
    Args:
        angle (np.ndarray): angles in degree or cartesian coordinates
        proba (np.ndarray): each angle's probability of occurrence
        type (str): "polar" or "cartesian"
    Usage:
        .. code-block:: python
            import numpy as np
            from bsfit.nodes.cirpy.utils import get_circ_weighted_mean_std
            degree = np.array([358, 0, 2, 88, 90, 92])
            proba = np.array([1, 1, 1, 1, 1, 1])/6
            output = get_circ_weighted_mean_std(degree, proba, "polar")
            output.keys()
            # Out: dict_keys(['coord_all', 'deg_all', 'coord_mean', 'deg_mean',
            #               'deg_all_for_std', 'deg_mean_for_std', 'deg_var',
            #               'deg_std', 'deg_sem'])
            output["deg_mean"]
            # Out: array([45.])
            output["deg_std"]
            # array([45.02961988])
    Returns:
        (dict): angle summary statistics (mean, std, var, sem)

    Raises:
        ValueError: type is not "polar" or "cartesian"
    """

    angle = angle.copy()

    # if polar, convert to cartesian
    if type == "polar":
        radius = 1
        coord = get_polar_to_cartesian(angle, radius=radius, type="polar")
    elif type == "cartesian":
        coord = angle
    else:
        raise ValueError(""" "type" can either be "polar" or "cartesian" value """)

    # store angles
    data = dict()
    data["coord_all"] = coord["cart"]
    data["deg_all"] = coord["deg"]

    # calculate mean
    # ..............
    proba_for_mean = np.tile(proba[:, None], 2)
    data["coord_mean"] = np.sum(proba_for_mean * data["coord_all"], 0)
    data["coord_mean"] = data["coord_mean"][:, None]
    data["deg_mean"] = get_cartesian_to_deg(
        data["coord_mean"][0],
        data["coord_mean"][1],
        signed=False,
    )

    # calculate std
    # ..............
    n_data = len(data["deg_all"])
    data["deg_all_for_std"] = data["deg_all"]
    data["deg_mean_for_std"] = np.tile(data["deg_mean"], n_data)

    # apply corrections
    # when 0 <= mean <= 180
    if data["deg_mean"] + 180 <= 360:
        for ix in range(n_data):
            if data["deg_all"][ix] >= data["deg_mean"] + 180:
                data["deg_all_for_std"][ix] = data["deg_all"][ix] - 360
    else:
        # when 180 <= mean <= 360
        for ix in range(n_data):
            if data["deg_all"][ix] <= data["deg_mean"] - 180:
                data["deg_mean_for_std"][ix] = data["deg_mean"] - 360

    # calculate variance, standard deviation and
    # standard error to the mean
    data["deg_var"] = np.array(
        [sum(proba * (data["deg_all_for_std"] - data["deg_mean_for_std"]) ** 2)]
    )
    data["deg_std"] = np.sqrt(data["deg_var"])
    data["deg_sem"] = data["deg_std"] / np.sqrt(n_data)
    return data


def get_signed_angle(origin: np.ndarray, destination: np.ndarray, type: str):
    """get the signed angle difference between origin and destination angles
    Args:
        origin (np.ndarray): origin angle
        destination (np.ndarray): destination angle
        type (str): angle type ("polar", "radian", "cartesian")
    Usage:
        .. code-block:: python
            angle = get_signed_angle(90, 45, 'polar')

            # Out: array([45.])

            angle = get_signed_angle(90, 45, 'radian')
            # Out: array([58.3103779])
            origin = np.array([[0, 1]])
            destination = np.array([[1, 0]])
            angle = get_signed_angle(origin, destination, "cartesian")

            # Out: array([90.])
    Returns:
        (np.ndarray): signed angle differences
    """

    # convert to cartesian coordinates
    if type == "polar" or type == "radian":
        origin_dict = get_polar_to_cartesian(origin, radius=1, type=type)
        destination_dict = get_polar_to_cartesian(destination, radius=1, type=type)
    elif type == "cartesian":
        origin_dict = dict()
        destination_dict = dict()
        origin_dict["cart"] = origin
        destination_dict["cart"] = destination

    # get coordinates
    xV1 = origin_dict["cart"][:, 0]
    yV1 = origin_dict["cart"][:, 1]
    xV2 = destination_dict["cart"][:, 0]
    yV2 = destination_dict["cart"][:, 1]

    # Calculate the angle separating the
    # two vectors in degrees
    angle = -(180 / np.pi) * np.arctan2(xV1 * yV2 - yV1 * xV2, xV1 * xV2 + yV1 * yV2)
    return angle


def get_combination_set(database: np.ndarray):
    """get the set of row combinations

    Args:
        database (np.ndarray): an N-D array

    Returns:
        (np.ndarray, np.ndarray, np.ndarray): `combs` is the set
        of combinations, `b` are the row indices for each combination
        in database, `c` are the rows indices for each combination in
        combs.
    """
    combs, ia, ic = np.unique(
        database,
        return_index=True,
        return_inverse=True,
        axis=0,
    )
    return (combs, ia, ic)


def get_data_stats(data: pd.Series, output: dict):
    """calculate data statistics

    Args:
        data (pd.Series): stimulus feature estimates
        output (dict): ::

            'PestimateGivenModel': estimate probabilities
            'map': max-a-posteriori percepts
            'conditions': task conditions

    Returns:
        (dict): returns data mean and std to output
    """
    # get conditions
    cond = output["conditions"]

    # initialise statistics
    data_mean = []
    data_std = []

    # get set of conditions
    cond_set, ix, _ = get_combination_set(cond)

    # record stats by condition
    for c_i in range(len(cond_set)):
        # find condition's instances
        loc_1 = cond[:, 0] == cond_set[c_i, 0]
        loc_2 = cond[:, 1] == cond_set[c_i, 1]
        loc_3 = cond[:, 2] == cond_set[c_i, 2]

        # get associated data
        data_c_i = data.values[loc_1 & loc_2 & loc_3]

        # set each instance with equal probability
        trial_proba = np.tile(1 / len(data_c_i), len(data_c_i))

        # get statistics
        stats = get_circ_weighted_mean_std(
            data_c_i,
            trial_proba,
            type="polar",
        )

        # record statistics
        data_mean.append(stats["deg_mean"])
        data_std.append(stats["deg_std"])

    # record statistics
    output["data_mean"] = np.array(data_mean)
    output["data_std"] = np.array(data_std)

    # record their condition
    output["conditions"] = cond_set
    return output


# Visualization utils
# -------------------
def plot_mean(
    data_mean: np.ndarray,
    data_std: np.ndarray,
    condition: np.ndarray,
    prior_mode: float,
    centering: bool,
):
    """plot data and prediction mean and std
    for three conditions (x-axis, colors and panels)
    Args:
        data_mean (np.ndarray): data mean by condition
        data_std (np.ndarray): data std by condition
        prediction_mean (np.ndarray): prediction mean by condition
        prediction_std (np.ndarray): prediction std by condition
        condition (np.ndarray): associated conditions
        prior_mode (float): the mode of the prior
        centering (bool): center x-axis or not
    Returns:
        _type_: _description_
    """
    # get condition levels
    levels_1 = np.flip(np.unique(condition[:, 0]))
    levels_2 = np.flip(np.unique(condition[:, 1]))  # sort in decreasing order
    levels_3 = np.unique(condition[:, 2])

    # set x_tick
    x_tick_centered = get_signed_angle(levels_1, prior_mode, "polar")
    x_tick_centered[x_tick_centered == -180] = 180
    i_sort = np.argsort(x_tick_centered)
    x_tick_centered = x_tick_centered[i_sort]
    y_tick_centered = x_tick_centered

    # set colors
    levels_2_color = [
        [0.5, 0, 0],
        [1, 0.2, 0],
        [1, 0.6, 0],
        [0.75, 0.75, 0],
    ]

    plt.figure(figsize=(10, 3))

    # loop over conditions and plot data
    # and prediction stats
    for level2_ix in range(len(levels_2)):
        # set condition 2 in column panels
        plt.subplot(1, len(levels_2), level2_ix + 1)

        # set condition 3 within panels
        for level1_ix in range(len(levels_1)):
            # find condition's instances
            loc_lev1 = condition[:, 0] == levels_1[level1_ix]
            loc_lev2 = condition[:, 1] == levels_2[level2_ix]
            loc_condition = loc_lev2 & loc_lev1

            # center to prior mode
            x_centered = condition[:, 2][loc_condition]
            if centering:
                x_centered = np.round(
                    get_signed_angle(
                        x_centered,
                        prior_mode,
                        "polar",
                    )
                )

            x_centered[x_centered == -180] = 180

            # make 2-D array
            x_centered = x_centered[:, None]

            # sort data stats
            y_data_centered = data_mean[loc_condition]
            y_data_std_centered = data_std[loc_condition]

            # sort all
            i_sort = np.argsort(x_centered.squeeze())
            x_centered = x_centered[i_sort]
            y_data_centered = y_data_centered[i_sort]
            y_data_std_centered = y_data_std_centered[i_sort]

            # To plot estimates mean against circular stimulus
            # feature on a linear space, the raw stimulus feature and
            # estimates mean are normalized to vectorial angles from
            # the prior mode and x and y axes are centered at zero
            # (normalized prior mode) via a circular shift. Rotation
            # angles were then labelled according to their raw values
            # on the circle (e.g., 0, is labelled 225). A mean estimate
            # of 33 degree was calculated for 55 degree which is very far
            # from stimulus feature on the linear space but actually close
            # to stimulus feature on the circular space. We got rid of
            # this visual artifact by expressing both 55 and 33 as the
            # counterclockwise distance to prior mode (e.g., for a prior
            # mode 225 55 becomes 190 instead of 170 and 33 becomes 168).
            # Note that the maximum vectorial angle is >180.
            if (level2_ix == 3) & (not 180 in x_centered):
                # move point at -170? distance to prior at 190? (positive
                # side) and convert values at x=-170? to positive distance
                # relative to prior to improve visualization
                posNeg170 = x_centered == -170
                x_centered[posNeg170] = prior_mode - 170 + 360 - prior_mode
                x_centered[x_centered == 180] = -180
                y_data_centered[posNeg170] = (
                    prior_mode - abs(y_data_centered[posNeg170]) + 360 - prior_mode
                )

                # sort x-axis
                i_sort = np.argsort(x_centered)
                x_centered = x_centered[i_sort]

                # sort y-axis
                y_data_centered = y_data_centered[i_sort]

                # set ticks
                x_tick_centered = x_centered

            # plot data stats
            plt.errorbar(
                x_centered.squeeze(),
                y_data_centered.squeeze(),
                yerr=y_data_std_centered.squeeze(),
                marker="o",
                markersize=7,
                markeredgecolor="w",
                linestyle="None",
                linewidth=1,
                color=levels_2_color[level1_ix],
                ecolor=levels_2_color[level1_ix],
            )
            plt.ylim([-0, 360])
            plt.hlines(prior_mode, -180, 180, linestyle=":")
            plt.xticks([-160, -80, 0, 80, 160], [-160, -80, 0, 80, 160])
            plt.yticks([67, 147, 227, 307, 387], [-160, -80, 0, 80, 160])
            plt.xlabel(
                "Motion direction distance"
                "\n"
                "relative to the prior mean"
                "\n"
                "(deg)"
            )
            if level2_ix == 0:
                plt.ylabel(
                    "Mean estimates distance"
                    "\n"
                    "relative to the prior mean"
                    "\n"
                    "(deg)"
                )
        plt.title(f"{int(levels_2[level2_ix]*100)}% coherence", fontsize=12)
    plt.show()
    return None

In [48]:
model_data

[(3.9269908169872414, 3.937168066555989, 0.12, 10, 3.9269908169872414),
 (3.9269908169872414, 3.897168066555993, 0.12, 10, 3.9269908169872414),
 (4.101523742186674, 4.037168066555992, 0.06, 10, 3.9269908169872414),
 (3.9269908169872414, 4.017168066555991, 0.06, 10, 3.9269908169872414),
 (3.752457891787808, 3.997168066555991, 0.24, 10, 3.9269908169872414),
 (3.5779249665883754, 3.897168066555993, 0.24, 10, 3.9269908169872414),
 (4.101523742186674, 3.95716806655599, 0.06, 10, 3.9269908169872414),
 (3.752457891787808, 3.837168066555989, 0.12, 10, 3.9269908169872414),
 (3.9269908169872414, 4.017168066555991, 0.12, 10, 3.9269908169872414),
 (3.9269908169872414, 4.117168066555992, 0.12, 10, 3.9269908169872414),
 (3.9269908169872414, 4.017168066555991, 0.06, 10, 3.9269908169872414),
 (4.101523742186674, 3.95716806655599, 0.12, 10, 3.9269908169872414),
 (3.9269908169872414, 3.897168066555993, 0.24, 10, 3.9269908169872414),
 (3.9269908169872414, 3.997168066555991, 0.06, 10, 3.9269908169872414),

In [10]:
def preprocess_data(df):
    # Convert motion direction and estimates to radians
    df["motion_direction_rad"] = get_deg_to_rad(df["motion_direction"], False)
    df["prior_mean_rad"] = np.deg2rad(df["prior_mean"])

    df["estimate_angle_rad"] = get_deg_to_rad(
        get_cartesian_to_deg(df["estimate_x"], df["estimate_y"], False), False
    )
    # df["estimate_angle"] = get_cartesian_to_deg(df["estimate_x"], df["estimate_y"], False)

    # Create a list of tuples (true_direction, estimated_direction, coherence, prior_std, prior_mean)
    data_for_model = list(
        zip(
            df["motion_direction_rad"],
            df["estimate_angle_rad"],
            df["motion_coherence"],
            df["prior_std"],
            df["prior_mean_rad"],
        )
    )

    return data_for_model


# Usage example:
model_data = preprocess_data(data)

In [51]:
# Now you can use this data with the updated BayesianObserverModel
model = BayesianObserverModel(
    kappa_e_low=1,
    kappa_e_med=2,
    kappa_e_high=3,
    kappa_prior_low=0.5,
    kappa_prior_med=1,
    kappa_prior_high=1.5,
    kappa_prior_high_coh=2,
    kappa_m=1,
    p_r=0.1,
)
fitted_params = model.fit(model_data)
print("Fitted parameters:", fitted_params)

ValueError: in1 and in2 should have the same dimensionality